# Sistemas de Recomendación

<div align="center"><a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/MachineLearning/11_Recomendacion/sistemas_recomendacion_sol.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg'/> </a> <br> Recordá abrir en una nueva pestaña </div>

In [1]:
import pandas as pd
import numpy as np

## Similitud coseno

$$sim(\pmb x, \pmb y) = \frac {\pmb x \cdot \pmb y}{||\pmb x|| \cdot ||\pmb y||}$$

¿Cómo calcularla en Python?

Supongamos que tenemos la siguiente matriz:

|  	| Libro A 	| Libro B 	| Libro C 	|
|-------	|---------	|---------	|---------	|
| Juan 	| 5 	| 4 	| 4 	|
| Diego 	| 4 	| 5 	| 5 	|


Podemos calcular la similitud coseno empleando sklearn:

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
Juan = [5,4,4]
Diego = [4,5,5]
cosine_similarity([Juan, Diego])

array([[1.        , 0.97823198],
       [0.97823198, 1.        ]])

También podemos calcular la similitud a mano:

In [3]:
(5*4 + 4*5 + 4*5)/(np.sqrt(5**2+4**2+4**2)*np.sqrt(4**2+5**2+5**2))

np.float64(0.9782319760890369)

O empleando Numpy

Calcular la similitud coseno usando numpy (con np.dot y np.linalg.norm)

In [4]:
np.dot(Juan,Diego)/np.dot(np.linalg.norm(Juan), np.linalg.norm(Diego))

np.float64(0.9782319760890369)

Ahora bien, cuando tenemos una matriz user-item de la vida real, tenemos muchos casos faltantes. En esta situación, no podremos calcular la similitud coseno tan fácilmente...

In [5]:
user_item = np.array([[5, np.nan, 4],[4,3,5],[4,5,5],[np.nan, 5, np.nan], [np.nan, 5, 3]])
user_item

array([[ 5., nan,  4.],
       [ 4.,  3.,  5.],
       [ 4.,  5.,  5.],
       [nan,  5., nan],
       [nan,  5.,  3.]])

# Recomendación basada en el contenido

En este ejemplo vamos a tomar un corpus de textos de autores latinoamericanos para sugerir uno similar a uno dado. Para esto construiremos una matriz TFIDF, de frecuencias normalizadas de términos por documento, y usaremos la similitud coseno para medir distancias entre los distintos textos.

In [6]:
!git clone https://github.com/karen-pal/borges

Cloning into 'borges'...
remote: Enumerating objects: 328, done.
remote: Counting objects: 100% (328/328), done.
remote: Compressing objects: 100% (264/264), done.
remote: Total 328 (delta 131), reused 249 (delta 60), pack-reused 0 (from 0)
Receiving objects: 100% (328/328), 26.76 MiB | 8.98 MiB/s, done.
Resolving deltas: 100% (131/131), done.


In [7]:
import pickle
from pathlib import Path
import pandas as pd

df = pd.DataFrame()
# usando el asterisco de "wildcard" traemos todos los archivos en formato pickle

pkls = Path('.').glob('./borges/datasets/datasets_pkl/*texts.pkl')

# leemos todos los pickles y concatenarlos en un DataFrame
for pkl in pkls:
    with open(pkl, 'rb') as inp:
        df_ = pickle.load(inp)
    df = pd.concat([df, df_])

df.shape

(719, 3)

In [8]:
df.sample(2)

,link,text_metadata,text
2,https://ciudadseva.com/texto/la-senorita-julia/,"{'title': 'La señorita Julia', 'metadata': '[C...","La señorita Julia, como la llamaban sus compañ..."
0,https://ciudadseva.com/texto/083/,"{'title': '83', 'metadata': '[Minicuento - Tex...",Luder pasa rápidamente delante de un mendigo q...


In [9]:
# separamos de la metadata el título y autor en sus propias columnas
df['title'] = df['text_metadata'].apply(lambda x: x['title'])
df['author'] = df['text_metadata'].apply(lambda x: x['author'])

In [10]:
# vemos los autores disponibloes
df['author'].value_counts()

,count
author,
Jorge Luis Borges,60
Julio Cortázar,55
Baldomero Lillo,50
Augusto Monterroso,45
Juan José Arreola,45
Alfonso Reyes,37
Enrique Anderson Imbert,36
Mario Benedetti,33
Julio Ramón Ribeyro,27


In [11]:
# quitamos duplicados y reiniciamos el índice
df = df.drop_duplicates(subset=[c for c in df.columns if c != 'text_metadata'])
df = df.reset_index(drop=True)
df.shape

(693, 5)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from pprint import pprint

Vamos a calcular las matrices de ocurrencias de términos usando sklearn.

Ámbas clases primero construyen el vocabulario total, y luego:  
- **CountVectorizer** nos devuelve la frecuencia absoluta de cada término por cada documento.
- [**TF-IDF**](https://en.wikipedia.org/wiki/Tf%E2%80%93idf): calcula la frecuencia de cada término por documento, y normaliza por el total de documentos donde el término aparece.

$${tf} (t,d)={\frac {f_{t,d}}{\sum _{t'\in d}{f_{t',d}}}}$$

$$
idf( t, D ) = log \frac{ \text{| } D \text{ |} }{ 1 + \text{| } \{ d \in D : t \in d \} \text{ |} }
$$


$$ tfidf( t, d, D ) = tf( t, d ) \times idf( t, D )
$$


In [13]:
# Instanciamos el CV
vectorizer = CountVectorizer()

doc1 = 'la matriz de frecuencias por palabras otorga información del contenido de un documento'
doc2 = 'las palabras que aparecen en un documento se relaciona con su tema'
# Definimos una lista con todos los strings
data_corpus = [doc1, doc2]

# Fiteamos el CV y transformamos los datos
X = vectorizer.fit_transform(data_corpus)

# Pasamos de sparse matrix a array usando .toarray()

print(X.toarray())
# Usando el metodo .get_feature_names() del CV podemos acceder al indice de palabras

print(vectorizer.get_feature_names_out())

[[0 0 1 2 1 1 0 1 1 1 0 1 1 1 1 0 0 0 0 0 1]
 [1 1 0 0 0 1 1 0 0 0 1 0 0 1 0 1 1 1 1 1 1]]
['aparecen' 'con' 'contenido' 'de' 'del' 'documento' 'en' 'frecuencias'
 'información' 'la' 'las' 'matriz' 'otorga' 'palabras' 'por' 'que'
 'relaciona' 'se' 'su' 'tema' 'un']


In [14]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop = list(stopwords.words('spanish'))
# eliminamos las "stop words", palabras comunes no informativas
tf = TfidfVectorizer(stop_words=stop)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
# calculamos los features para cada ítem (texto)
tfidf_matrix = tf.fit_transform(df['text'])

In [16]:
# calculamos las similitudes entre todos los documentos
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
n = 6

# diccionario creado para guardar el resultado en un formato (autor - titulo : puntaje, titulo, autor)
results = {}
for idx, row in df.iterrows():
    # guardamos los indices similares basados en la similitud coseno. Los ordenamos en modo ascendente, siendo 0 nada de similitud y 1 total
    similar_indices = cosine_similarities[idx].argsort()[:-n-2:-1]
    # guardamos los N más cercanos
    similar_items = [(f"{df['author'][i]} - {df['title'][i]}", round(cosine_similarities[idx][i], 3)) for i in similar_indices]
    results[f"{row['author']} - {row['title']}"] = similar_items[1:]

In [17]:
pprint(results['Jorge Luis Borges - El Aleph'])

[('Jorge Luis Borges - La escritura del dios', np.float64(0.144)),
 ('Jorge Luis Borges - El inmortal', np.float64(0.135)),
 ('Jorge Luis Borges - Utopía de un hombre que está cansado',
  np.float64(0.125)),
 ('Felisberto Hernández - El acomodador', np.float64(0.122)),
 ('Clarice Lispector - La búsqueda de la dignidad', np.float64(0.121)),
 ('Jorge Luis Borges - Funes el memorioso', np.float64(0.11))]


In [18]:
def recomendar(autor, titulo):
    pprint(results[f"{autor} - {titulo}"])

In [19]:
recomendar('Julio Cortázar', 'Axolotl')

[('Felisberto Hernández - El acomodador', np.float64(0.134)),
 ('Felisberto Hernández - El cocodrilo', np.float64(0.101)),
 ('Felisberto Hernández - Menos Julia', np.float64(0.089)),
 ('Julio Cortázar - Después del almuerzo', np.float64(0.088)),
 ('Julio Cortázar - La noche boca arriba', np.float64(0.086)),
 ('Julio Cortázar - La señorita Cora', np.float64(0.086))]
